In [1]:
import sys
import os

import pandas as pd

sys.path.append(os.path.abspath('..'))
from src.env import PACKAGE_DIR
from configulator.config import default_config
from src.utils.call_api.schema import EmbeddingEngineEnum
from src.utils.preprocess.text_splitter import CharTextSplitter
from src.retrievers.models.bm25_retriever import KeywordRetriever
from src.retrievers.models.vector_retriever import VectorRetriever
from src.retrievers.models.ensemble_retriever import EnsembleRetriever
from src.evaluator.eval_retriever.evaluator import RetieverEvaluator

pd.set_option("display.max_colwidth", 500)
pd.set_option('display.max_columns', None)

In [2]:
cfg = default_config()
cfg.chunksize = 200
cfg.overlap = 0.2
NOVEL_DIR = PACKAGE_DIR/'data/processed/valide_sets/novels'
QUERY_FILE = PACKAGE_DIR/'data/raw/valid_sets/query_ans_txt.csv'

In [3]:
# splitter = CharTextSplitter(NOVEL_DIR, cfg.chunksize, cfg.overlap, from_tiktoken=False)
# index_df = splitter.split_novels(return_df=True)
query_df = pd.read_csv(QUERY_FILE)

In [4]:
index_df = pd.read_csv(PACKAGE_DIR/'data/database/valid_sets/chartext_chunk200_lap20/openai_small.csv')

In [5]:
keyretriever = KeywordRetriever(index_df)
vecretriever = VectorRetriever(index_df, model=EmbeddingEngineEnum.Small)
ensretriever = EnsembleRetriever(vecretriever, keyretriever)

100%|██████████| 108/108 [00:00<00:00, 4007.76it/s]


In [6]:
def eval_retriever(retriever, top_k, query_df, return_miss: bool = False, weights = [0.5, 0.5], ensemble_method = 'rrf', rank_impact_mitigator=60):
    retrieved_texts_list = []
    for query in query_df['problem'].values:
        if isinstance(retriever, EnsembleRetriever):
            retrieved_texts = retriever.retrieve(query, weights=weights, ensemble_method=ensemble_method, rank_impact_mitigator=rank_impact_mitigator, top_k=top_k)
        else:
            retrieved_texts = retriever.retrieve(query, top_k=top_k)
        retrieved_texts_list.append(retrieved_texts['similar_document'])
    evaluator = RetieverEvaluator(query_df['evidence'], retrieved_texts_list)
    recall_score = evaluator.compute_mean_recall()

    if return_miss:
        return recall_score, evaluator.failed_retrieved_evidence

    return recall_score

In [13]:
eval_retriever(keyretriever, 25, query_df, return_miss=True)

(0.6428571428571429,
 ['吉田は肺が悪い',
  '人間の脳味噌の黒焼き',
  '鼠の仔を捕って来て入れてそれを黒焼きにしたもの',
  'いきなりその眼の前へ顔を近付けて、非常に押し殺した力強い声で、「心臓へ来ましたか？」と耳打ちをした女があった',
  '「その病気に利くええ薬を教えたげまひょか」',
  'ひょっとしてあなたは肺がお悪いのじゃありませんか',
  '吉田はそんな話を聞くにつけても、そういう迷信を信じる人間の無智に馬鹿馬鹿しさを感じないわけにいかなかったけれども、考えてみれば人間の無智というのはみな程度の差で、そう思って馬鹿馬鹿しさの感じを取り除いてしまえば、あとに残るのはそれらの人間の感じている肺病に対する手段の絶望と、病人達のなんとしてでも自分のよくなりつつあるという暗示を得たいという二つの事柄なのであった'])

In [14]:
eval_retriever(vecretriever, 25, query_df, return_miss=True)

(0.7857142857142857,
 ['人間の脳味噌の黒焼き',
  '首縊りの縄',
  '母親の話によるとそれは青物を売りに来る女があって、その女といろいろ話をしているうちにその肺病の特効薬の話をその女がはじめたというのだった',
  'その女には肺病の弟があってそれが死んでしまった',
  'ひょっとしてあなたは肺がお悪いのじゃありませんか',
  '吉田はそんな話を聞くにつけても、そういう迷信を信じる人間の無智に馬鹿馬鹿しさを感じないわけにいかなかったけれども、考えてみれば人間の無智というのはみな程度の差で、そう思って馬鹿馬鹿しさの感じを取り除いてしまえば、あとに残るのはそれらの人間の感じている肺病に対する手段の絶望と、病人達のなんとしてでも自分のよくなりつつあるという暗示を得たいという二つの事柄なのであった'])

In [10]:
for vec in [0.1, 0.2, 0.3, 0.4, 0.5]:
    score = eval_retriever(ensretriever, 50, query_df, weights=[vec, 1-vec], ensemble_method='rrf', rank_impact_mitigator=60)
    print(f'score is {score} when vec: {vec}, key: {1-vec}')

score is 0.7142857142857143 when vec: 0.1, key: 0.9
score is 0.7380952380952381 when vec: 0.2, key: 0.8
score is 0.7857142857142857 when vec: 0.3, key: 0.7
score is 0.7857142857142857 when vec: 0.4, key: 0.6
score is 0.7380952380952381 when vec: 0.5, key: 0.5


In [ ]:
q_id = 0
retriever = KeywordRetriever(index_df)
print(query_df.loc[q_id, ['problem', 'evidence']])
retriever.retrieve(query_df.loc[q_id, 'problem'], top_k=10, require_columns=['chunk_id']).sort_values(by=['chunk_id'])

100%|██████████| 108/108 [00:00<00:00, 2808.25it/s]

problem     小説「のんきな患者」で、主人公の吉田の患部は主にどこですか？
evidence                         「吉田は肺が悪い」
Name: 0, dtype: object


,similar_document,similarity,chunk_id
8,しかしそれがやっとのことで成功したと思うと、方向を変えた猫は今度はのそのそと吉田の寝床の上へあがってそこで丸くなって毛を舐めはじめた。そこへ行けばもう吉田にはどうすることもできない場所である。薄氷を踏むような吉田の呼吸がにわかにずしりと重くなった。吉田はいよいよ母親を起こそうかどうしようかということで抑えていた癇癪を昂ぶらせはじめた,1.804980,21
0,吉田は何かきっとそれは自分の寐つく前に読んだ小説かなにかのなかにあったことにちがいないと思うのだったがそれが思い出せなかった。また吉田は「自己の残像」というようなものがあるものなんだなというようなことを思ったりした,7.269377,28
3,しかし吉田は決してほかを向いて寐ようという気はしなかった。そうするとせっかく自分の感じている春の夜のような気持が一時に病気病気した冬のような気持になってしまうのだった。しかし寐られないということも吉田にとっては苦痛であった。吉田はいつか不眠症ということについて、それの原因は結局患者が眠ることを欲しないのだという学説があることを人に聞かされていた,4.360530,32
2,通り筋の毛糸雑貨屋の主人はこの間まで店へ据えた毛糸の織機で一日中毛糸を織っていたが、急に死んでしまって、家族がすぐ店を畳んで国へ帰ってしまったそのあとはじきカフエーになってしまった,4.446395,66
5,この町へ帰って来てしばらくしてから吉田はまた首縊りの縄を「まあ馬鹿なことやと思うて」嚥んでみないかと言われた。それをすすめた人間は大和で塗師をしている男でその縄をどうして手に入れたかという話を吉田にして聞かせた。\nそれはその町に一人の鰥夫の肺病患者があって、その男は病気が重ったままほとんど手当をする人もなく、一軒の荒ら家に捨て置かれてあったのであるが、とうとう最近になって首を縊って死んでしまった,3.662912,75
4,はっとして吉田がその女の顔を見ると、それはその病舎の患者の付添いに雇われている付添婦の一人で、勿論そんな付添婦の顔触れにも毎日のように変化はあったが、その女はその頃露悪的な冗談を言っては食堂へ集まって来る他の付添婦たちを牛耳っていた中婆さんなのだった。\n吉田はそう言われて何のことかわからずにしばらく相手の顔を見ていたが、すぐに「ああなるほど」と気のついたことがあった,3.747455,80
1,吉田は病気のためにたまにこうした機会にしか直接世間に触れることがなかったのであるが、そしてその触れた世間というのはみな吉田が肺病患者だということを見破って近付いて来た世間なのであるが、病院にいる一と月ほどの間にまた別なことに打つかった。\nそれはある日吉田が病院の近くの市場へ病人の買物に出かけたときのことだった,4.466132,87
9,それはある日吉田が病院の近くの市場へ病人の買物に出かけたときのことだった。吉田がその市場で用事を足して帰って来ると往来に一人の女が立っていて、その女がまじまじと吉田の顔を見ながら近付いて来て、\n「もしもし、あなた失礼ですが……」\nと吉田に呼びかけたのだった,1.793147,88
6,勿論これは単に「肺結核によって死んだ人間」の統計で肺結核に対する極貧者の死亡率や上流階級の者の死亡率というようなものを意味していないので、また極貧者と言ったり上流階級と言ったりしているのも、それがどのくらいの程度までを指しているのかはわからないのであるが、しかしそれは吉田に次のようなことを想像せしめるには充分であった。\nつまりそれは、今非常に多くの肺結核患者が死に急ぎつつある,3.424441,105
7,つまりそれは、今非常に多くの肺結核患者が死に急ぎつつある。そしてそのなかで人間の望み得る最も行き届いた手当をうけている人間は百人に一人もないくらいで、そのうちの九十何人かはほとんど薬らしい薬ものまずに死に急いでいるということであった,3.149211,106


In [20]:
vecretriever = VectorRetriever(index_df, model=EmbeddingEngineEnum.Large)
vecretriever.retrieve('主人公の吉田の患部は主にどこですか', top_k=10, require_columns=['chunk_id'])

,similar_document,similarity,chunk_id
0,そんなふうではじめ吉田にはその娘のことよりもお婆さんのことがその荒物屋についての知識を占めていたのであるが、だんだんその娘のことが自分のことにも関聯して注意されて来たのはだいぶんその娘の容態も悪くなって来てからであった。近所の人の話ではその荒物屋の親爺さんというのが非常に吝嗇で、その娘を医者にもかけてやらなければ薬も買ってやらないということであった,0.063045,51
1,病勢がこんなになるまでの間、吉田はこれを人並みの流行性感冒のように思って、またしても「明朝はもう少しよくなっているかもしれない」と思ってはその期待に裏切られたり、今日こそは医者を頼もうかと思ってはむだに辛抱をしたり、いつまでもひどい息切れを冒しては便所へ通ったり、そんな本能的な受身なことばかりやっていた,0.059192,3
2,その原因を臆測するにもまたその正否を判断するにも結局当の自分の不安の感じに由るほかはないのだとすると、結局それは何をやっているのかわけのわからないことになるのは当然のことなのだったが、しかしそんな状態にいる吉田にはそんな諦めがつくはずはなく、いくらでもそれは苦痛を増していくことになるのだった。\n第二に吉田を苦しめるのはこの不安には手段があると思うことだった,0.057148,8
3,しかし母親を起こすことを考えると、こんな感情を抑えておそらく何度も呼ばなければならないだろうという気持だけでも吉田はまったく大儀な気になってしまうのだった。――しばらくして吉田はこの間から自分で起こしたことのなかった身体をじりじり起こしはじめた。そして床の上へやっと起きかえったかと思うと、寝床の上に丸くなって寝ている猫をむんずと掴まえた。吉田の身体はそれだけの運動でもう浪のように不安が揺れはじめた,0.055209,23
4,だから吉田の頭には地震とか火事とか一生に一度遭うか二度遭うかというようなものまでが真剣に写っているのだった。また吉田がこの状態を続けてゆくというのには絶えない努力感の緊張が必要であって、もしその綱渡りのような努力になにか不安の影が射せばたちどころに吉田は深い苦痛に陥らざるを得ないのだった。――しかしそんなことはいくら考えても決定的な知識のない吉田にはその解決がつくはずはなかった,0.053004,7
5,吉田の弟の店のあるところはその間でも比較的早くからできていた通り筋で両側はそんな町らしい、いろんなものを商う店が立ち並んでいた。\n吉田は東京から病気が悪くなってその家へ帰って来たのが二年あまり前であった。吉田の帰って来た翌年吉田の父はその家で死んで、しばらくして吉田の弟も兵隊に行っていたのから帰って来ていよいよ落ち着いて商売をやっていくことになり嫁をもらった,0.047068,44
6,かと思うと「不安や」「不安や」と弱々しい声を出して訴えることもある。そういうときはきまって夜で、どこから来るともしれない不安が吉田の弱り切った神経を堪らなくするのであった。\n吉田はこれまで一度もそんな経験をしたことがなかったので、そんなときは第一にその不安の原因に思い悩むのだった,0.041403,5
7,煙草を喫うも喫わないも、その道具の手の届くところへ行きつくだけでも、自分の今のこの春の夜のような気持は一時に吹き消されてしまわなければならないということは吉田も知っていた。そしてもしそれを一服喫ったとする場合、この何日間か知らなかったどんな恐ろしい咳の苦しみが襲って来るかということも吉田はたいがい察していた,0.040683,34
8,吉田はそのお婆さんからはいつも少し人の好過ぎるやや腹立たしい印象をうけていたのであるが、それはそのお婆さんがまたしても変な笑い顔をしながら近所のおかみさんたちとお喋りをしに出て行っては、弄りものにされている――そんな場面をたびたび見たからだった,0.040190,49
9,吉田は肺が悪い。寒になって少し寒い日が来たと思ったら、すぐその翌日から高い熱を出してひどい咳になってしまった。胸の臓器を全部押し上げて出してしまおうとしているかのような咳をする。四五日経つともうすっかり痩せてしまった。咳もあまりしない。しかしこれは咳が癒ったのではなくて、咳をするための腹の筋肉がすっかり疲れ切ってしまったからで、彼らが咳をするのを肯じなくなってしまったかららしい,0.040149,1
